In [1]:
# import package
# installed via pip
from emtracks.particle import * # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
from emtracks.interpolations import *
import matplotlib.animation as animation
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
import os

from joblib import Parallel, delayed
import multiprocessing
from tqdm.notebook import tqdm


datadir = "/home/shared_data/mao10,mao13_analysis"
plotdir = datadir+"plots/mao13maps7-16/"
rad10plotdir = '/home/shared_data/mao10,mao13_analysis/plots/mao10maps7-16rad/'
rad13plotdir = '/home/shared_data/mao10,mao13_analysis/plots/mao13maps7-16rad/'
reg10plotdir = '/home/shared_data/mao10,mao13_analysis/plots/mao10maps7-16/'
reg13plotdir = '/home/shared_data/mao10,mao13_analysis/plots/mao13maps7-16/'

maomapdir = '/home/shared_data/Bmaps/Mau10/combined/'

mao13datadir = '/home/shared_data/mao10,mao13_analysis/data/mao13no_nom/'
mao10datadir = '/home/shared_data/mao10,mao13_analysis/data/mao10no_nom/'

ERROR! Please set $EMTRACKS_DDIR and $EMTRACKS_PDIR. Setting defaults (current directory)


In [2]:
def readpklold(zstart, zend, numpoints):
    files = sorted(os.listdir(mao13datadir))
    zsteps = np.linspace(zstart, zend, numpoints)
    data = []
    deleted = []

    for file in files:
        x = file.split('_')
        field = x[0]
        e_solvernom = trajectory_solver.from_pickle(mao13datadir+file)
        phi = e_solvernom.init_conds.phi0
        theta = e_solvernom.init_conds.theta0
        for z in zsteps:
            if z > e_solvernom.dataframe.z.max() or z < e_solvernom.dataframe.z.min():
                data.append(8*[np.nan])
                deleted.append([e_solvernom.init_conds.theta0, e_solvernom.init_conds.phi0])
            else:
                info = interp_cole(e_solvernom.dataframe, z)
                x = info[1]
                y = info[2]
                r = tuple([(x**2 + y**2)**(1/2)])
                r2 = tuple([((x-0.054094482)**(2) + (y-0.03873037)**(2))**(1/2)])
                tuple1 = (z, field, theta, phi)
                
                nan = tuple([np.nan])

                data.append(tuple1 + info + r + r2)
                
    return data, deleted

In [4]:
x, deleted = readpklold(6, 13, 8)

In [5]:
df = pd.DataFrame(x, columns = ['z', 'field', 'theta', 'phi', 't', 'x', 'y', 'r_(0,0)', 'r_(x0,y0)'])

In [6]:
df1 = df[df['field']!='nom']
dfnew = df1.dropna()

In [7]:
dfnew

,z,field,theta,phi,t,x,y,"r_(0,0)","r_(x0,y0)"
0,6.0,0.00,0.000000,0.000000,3.702312e-11,0.054095,0.038729,0.066530,0.000002
1,7.0,0.00,0.000000,0.000000,3.372879e-09,0.062695,0.039419,0.074058,0.008628
2,8.0,0.00,0.000000,0.000000,6.708647e-09,0.063993,0.043384,0.077313,0.010938
3,9.0,0.00,0.000000,0.000000,1.004441e-08,0.068830,0.044672,0.082056,0.015889
4,10.0,0.00,0.000000,0.000000,1.338017e-08,0.063585,0.045135,0.077976,0.011449
...,...,...,...,...,...,...,...,...,...
213995,9.0,1.00,2.179881,6.283185,7.019242e-08,0.061440,0.504347,0.508076,0.465675
213996,10.0,1.00,2.179881,6.283185,7.478121e-08,0.270110,0.132188,0.300721,0.235365
213997,11.0,1.00,2.179881,6.283185,7.935644e-08,-0.156282,0.227842,0.276290,0.282881
213998,12.0,1.00,2.179881,6.283185,8.390741e-08,0.221115,0.464740,0.514660,0.457580


In [8]:
def difarrays(df):
    ms = []
    data = []
    missing = []
    for z in df['z'].unique():
        for field in df['field'].unique():
            xdifs = []
            ydifs = []
            rdifs = []
            for theta in df['theta'].unique():
                dfnew1 = df[(df['z']==z) & np.isclose(df['theta'], theta, 1e-1) & ((df['field']=='1.00'))]
                dfnew2 = df[(df['z']==z) & np.isclose(df['theta'], theta, 1e-1) & ((df['field']==field))]                        
                if len(np.array(dfnew1['phi'])) != len(np.array(dfnew2['phi'])):
                    missing.append([z, field, theta])
                elif(np.isclose(v, 0) for v in (np.array(dfnew1['phi']) - np.array(dfnew2['phi']))):
                    xdif = np.array(dfnew2['x']) - np.array(dfnew1['x'])
                    ydif = np.array(dfnew2['y']) - np.array(dfnew1['y'])
                    
                
                    for a in range(0, len(xdif), 1):
                        data.append([xdif[a], ydif[a], z, field, theta])
                    for a in xdif:
                        xdifs.append(a)
                    for b in ydif:
                        ydifs.append(b)
                        
                else:
                    print('mistake')
            ms.append([field, z, np.mean(xdifs), np.mean(ydifs), np.std(xdifs), np.std(ydifs)])
                
    datadf = pd.DataFrame(data, columns = ['xdif', 'ydif', 'z', 'field', 'theta'])
    msdf = pd.DataFrame(ms, columns = ['field', 'z', 'xmean', 'ymean', 'xstd', 'ystd'])
                
    return msdf, datadf, missing 

In [9]:
msdf, datadf, missing = difarrays(dfnew)

In [10]:
msdf

,field,z,xmean,ymean,xstd,ystd
0,0.00,6.0,-0.006998,-0.002312,0.073921,0.074162
1,0.10,6.0,-0.007238,-0.001245,0.075518,0.068425
2,0.20,6.0,-0.006387,-0.000242,0.074425,0.063252
3,0.30,6.0,-0.007193,0.000310,0.066126,0.058112
4,0.40,6.0,-0.007102,0.002767,0.057093,0.056667
...,...,...,...,...,...,...
83,0.60,13.0,0.003734,-0.000738,0.047526,0.058169
84,0.70,13.0,0.004121,-0.000310,0.044491,0.039858
85,0.80,13.0,0.003366,-0.001236,0.033786,0.020768
86,0.90,13.0,0.001526,-0.001051,0.015331,0.010623


In [11]:
dfmissing = pd.DataFrame(missing, columns = ['z', 'field', 'theta'])
dfmissing['theta'].values.min()

1.9234240736264039

In [12]:
def makeplots(df, z):
    fields = df['field'].unique()
    fig = plt.figure()
    ax1 = plt.subplot2grid((2,2), (0,0), rowspan=1, colspan=1)
    ax2 = plt.subplot2grid((2,2), (0,1), rowspan=1, colspan=1)
    ax3 = plt.subplot2grid((2,2), (1,0), rowspan=1, colspan=1)
    ax4 = plt.subplot2grid((2,2), (1,1), rowspan=1, colspan=1)

    df2 = df[df['z']==z]
    for field in fields:
        dfnew = df2[df2['field']==field]
        
        xmean = list(dfnew['xmean'])
        ymean = list(dfnew['ymean'])
        xstd = list(dfnew['xstd'])
        ystd = list(dfnew['ystd'])
        ax1.scatter(field, xmean, c='r', marker='s')
        ax1.set_title(f'(Dis X - 1.00 X) Mean Value at Different Fields', fontsize = 20, loc = 'right')
        ax1.set_xlabel(f'Field Map Label')
        ax1.set_ylabel(f'(Dis X - 1.00 X) Mean Value (meters)')
        
        ax2.scatter(field, ymean, c='r', marker='s')
        ax2.set_title(f'(Dis Y - 1.00 Y) Mean Value at Different Fields', fontsize = 20, loc = 'right')
        ax2.set_xlabel(f'Field Map Label')
        ax2.set_ylabel(f'(Dis Y - 1.00 Y) Mean Value (meters)')
        
        ax3.scatter(field, xstd, c='r', marker='s')
        ax3.set_title(f'(Dis X - 1.00 X) Std Value at Different Fields', fontsize = 20, loc = 'right')
        ax3.set_xlabel(f'Field Map Label')
        ax3.set_ylabel(f'(Dis X - 1.00 X) Std Value (meters)')
        
        ax4.scatter(field, ystd, c='r', marker='s')
        ax4.set_title(f'(Dis Y - 1.00 Y) Std Value at Different Fields', fontsize = 20, loc = 'right')
        ax4.set_xlabel(f'Field Map Label')
        ax4.set_ylabel(f'(Dis Y - 1.00 Y) Std Value (meters)')
    fig.suptitle(f'Mean, Std X, Y Displacements (Dis X, Y - 1.00 X, Y) at Z = {z}', fontsize = 30)
    fig.savefig(reg13plotdir+f'/Mean,Std_Scatter_Z={z}.png')

        

In [14]:
zvals = datadf['z'].unique()
fieldvals = datadf['field'].unique()

In [ ]:
for z in zvals:
    makeplots(msdf, z)

In [ ]:
datadf

In [ ]:
for field in datadf['field'].unique():
    fig = plt.figure()
    df1 = datadf[(datadf['z']==zvals[0]) & (datadf['field']==field)]
    df2 = datadf[(datadf['z']==zvals[1]) & (datadf['field']==field)]
    df3 = datadf[(datadf['z']==zvals[2]) & (datadf['field']==field)]
    df4 = datadf[(datadf['z']==zvals[3]) & (datadf['field']==field)]
    df5 = datadf[(datadf['z']==zvals[4]) & (datadf['field']==field)]
    df6 = datadf[(datadf['z']==zvals[5]) & (datadf['field']==field)]
    df7 = datadf[(datadf['z']==zvals[6]) & (datadf['field']==field)]
    df8 = datadf[(datadf['z']==zvals[7]) & (datadf['field']==field)]
    
    ax1 = plt.subplot2grid((3,3), (0,0), rowspan=1, colspan=1)
    ax1.hist(list(df1['xdif']), bins = 20, facecolor = 'blue')
    ax1.set_title(f'Z = {zvals[0]}')
    ax1.set_ylabel(f'Number of Occurences')
    ax1.set_xlabel(f'X Displacement (meters)')
    
    ax2 = plt.subplot2grid((3,3), (0,1), rowspan=1, colspan=1)
    ax2.hist(list(df2['xdif']), bins = 20, facecolor = 'blue')
    ax2.set_title(f'Z = {zvals[1]}')
    ax2.set_ylabel(f'Number of Occurences')
    ax2.set_xlabel(f'X Displacement (meters)')
    
    ax3 = plt.subplot2grid((3,3), (0,2), rowspan=1, colspan=1)
    ax3.hist(list(df3['xdif']), bins = 20, facecolor = 'blue')
    ax3.set_title(f'Z = {zvals[2]}')
    ax3.set_ylabel(f'Number of Occurences')
    ax3.set_xlabel(f'X Displacement (meters)')
    
    ax4 = plt.subplot2grid((3,3), (1,0), rowspan=1, colspan=1)
    ax4.hist(list(df4['xdif']), bins = 20, facecolor = 'blue')
    ax4.set_title(f'Z = {zvals[3]}')
    ax4.set_ylabel(f'Number of Occurences')
    ax4.set_xlabel(f'X Displacement (meters)')
    
    ax5 = plt.subplot2grid((3,3), (1,1), rowspan=1, colspan=1)
    ax5.hist(list(df5['xdif']), bins = 20, facecolor = 'blue')
    ax5.set_title(f'Z = {zvals[4]}')
    ax5.set_ylabel(f'Number of Occurences')
    ax5.set_xlabel(f'X Displacement (meters)')
    
    ax6 = plt.subplot2grid((3,3), (1,2), rowspan=1, colspan=1)
    ax6.hist(list(df6['xdif']), bins = 20, facecolor = 'blue')
    ax6.set_title(f'Z = {zvals[5]}')
    ax6.set_ylabel(f'Number of Occurences')
    ax6.set_xlabel(f'X Displacement (meters)')
    
    ax7 = plt.subplot2grid((3,3), (2,0), rowspan=1, colspan=1)
    ax7.hist(list(df7['xdif']), bins = 20, facecolor = 'blue')
    ax7.set_title(f'Z = {zvals[6]}')
    ax7.set_ylabel(f'Number of Occurences')
    ax7.set_xlabel(f'X Displacement (meters)')
    
    ax8 = plt.subplot2grid((3,3), (2,1), rowspan=1, colspan=1)
    ax8.hist(list(df8['xdif']), bins = 20, facecolor = 'blue')
    ax8.set_title(f'Z = {zvals[7]}')
    ax8.set_ylabel(f'Number of Occurences')
    ax8.set_xlabel(f'X Displacement (meters)')
    
    fig.suptitle(f'Histogram of X Displacement (Dis X - Nom X) for Field = {field}', fontsize = 20)
    fig.tight_layout(pad = 5.0)
    fig.savefig(reg13plotdir+f'/X_hist_{field}.png')

In [ ]:
for field in datadf['field'].unique():
    fig = plt.figure()
    df1 = datadf[(datadf['z']==zvals[0]) & (datadf['field']==field)]
    df2 = datadf[(datadf['z']==zvals[1]) & (datadf['field']==field)]
    df3 = datadf[(datadf['z']==zvals[2]) & (datadf['field']==field)]
    df4 = datadf[(datadf['z']==zvals[3]) & (datadf['field']==field)]
    df5 = datadf[(datadf['z']==zvals[4]) & (datadf['field']==field)]
    df6 = datadf[(datadf['z']==zvals[5]) & (datadf['field']==field)]
    df7 = datadf[(datadf['z']==zvals[6]) & (datadf['field']==field)]
    df8 = datadf[(datadf['z']==zvals[7]) & (datadf['field']==field)]
    
    ax1 = plt.subplot2grid((3,3), (0,0), rowspan=1, colspan=1)
    ax1.hist(list(df1['ydif']), bins = 20, facecolor = 'blue')
    ax1.set_title(f'Z = {zvals[0]}')
    ax1.set_ylabel(f'Number of Occurences')
    ax1.set_xlabel(f'Y Displacement (meters)')
    
    ax2 = plt.subplot2grid((3,3), (0,1), rowspan=1, colspan=1)
    ax2.hist(list(df2['ydif']), bins = 20, facecolor = 'blue')
    ax2.set_title(f'Z = {zvals[1]}')
    ax2.set_ylabel(f'Number of Occurences')
    ax2.set_xlabel(f'Y Displacement (meters)')
    
    ax3 = plt.subplot2grid((3,3), (0,2), rowspan=1, colspan=1)
    ax3.hist(list(df3['ydif']), bins = 20, facecolor = 'blue')
    ax3.set_title(f'Z = {zvals[2]}')
    ax3.set_ylabel(f'Number of Occurences')
    ax3.set_xlabel(f'Y Displacement (meters)')
    
    ax4 = plt.subplot2grid((3,3), (1,0), rowspan=1, colspan=1)
    ax4.hist(list(df4['ydif']), bins = 20, facecolor = 'blue')
    ax4.set_title(f'Z = {zvals[3]}')
    ax4.set_ylabel(f'Number of Occurences')
    ax4.set_xlabel(f'Y Displacement (meters)')
    
    ax5 = plt.subplot2grid((3,3), (1,1), rowspan=1, colspan=1)
    ax5.hist(list(df5['ydif']), bins = 20, facecolor = 'blue')
    ax5.set_title(f'Z = {zvals[4]}')
    ax5.set_ylabel(f'Number of Occurences')
    ax5.set_xlabel(f'Y Displacement (meters)')
    
    ax6 = plt.subplot2grid((3,3), (1,2), rowspan=1, colspan=1)
    ax6.hist(list(df6['ydif']), bins = 20, facecolor = 'blue')
    ax6.set_title(f'Z = {zvals[5]}')
    ax6.set_ylabel(f'Number of Occurences')
    ax6.set_xlabel(f'Y Displacement (meters)')
    
    ax7 = plt.subplot2grid((3,3), (2,0), rowspan=1, colspan=1)
    ax7.hist(list(df7['ydif']), bins = 20, facecolor = 'blue')
    ax7.set_title(f'Z = {zvals[6]}')
    ax7.set_ylabel(f'Number of Occurences')
    ax7.set_xlabel(f'Y Displacement (meters)')
    
    ax8 = plt.subplot2grid((3,3), (2,1), rowspan=1, colspan=1)
    ax8.hist(list(df8['ydif']), bins = 20, facecolor = 'blue')
    ax8.set_title(f'Z = {zvals[7]}')
    ax8.set_ylabel(f'Number of Occurences')
    ax8.set_xlabel(f'Y Displacement (meters)')
    
    fig.suptitle(f'Histogram of Y Displacement (Dis Y - Nom Y) for Field = {field}', fontsize = 20)
    fig.tight_layout(pad = 5.0)
    fig.savefig(reg13plotdir+f'/Y_hist_{field}.png')

In [ ]:
for field in fieldvals:
    newdf = datadf[datadf['field']==field]
    fig = plt.figure()
    plt.scatter(newdf['z'], newdf['xdif'], c = 'blue')
    plt.title(f'X Deviations (Dis X - Nom X) for Field = {field} at Different Z Values')
    plt.xlabel('Z Value (meters)')
    plt.ylabel('X Deviation (meters)')
    fig.savefig(reg13plotdir+f'/deviations_{field}.png')

In [ ]:
def readradiuspkl(zstart, zend, numpoints):
    files = sorted(os.listdir(newdir))
    zsteps = np.linspace(zstart, zend, numpoints)
    data = []
    deleted = []

    for file in files:
        x = file.split('_')
        field = x[0]
        e_solvernom = trajectory_solver.from_pickle(newdir+file)
        phi = e_solvernom.init_conds.phi0
        theta = e_solvernom.init_conds.theta0
        for z in zsteps:
            if z > e_solvernom.dataframe.z.max() or z < e_solvernom.dataframe.z.min():
                data.append(9*[np.nan])
                deleted.append([e_solvernom.init_conds.theta0, e_solvernom.init_conds.phi0])
            elif z > 7.5:
                info = interp_cole(e_solvernom.dataframe, z)
                x = info[1]
                y = info[2]
                r = tuple([(x**2 + y**2)**(1/2)])
                r2 = tuple([((x-0.054094482)**(2) + (y-0.03873037)**(2))**(1/2)])
            
                df6 = e_solvernom.dataframe
                df5 = df6[(df6['z'] > 7.5) & (df6['z'] < 12.5)]
                xl = df5['x'].values
                yl = df5['y'].values
                zlist = df5['z'].values
                e = np.where(zlist < z)
                _, _, r3, _ = reco_circle(xl, yl)
                rreco = tuple([r3[e[0][-1]]])
                tuple1 = (z, field, theta, phi)

                data.append(tuple1 + info + r + r2 + rreco)
            else:
                info = interp_cole(e_solvernom.dataframe, z)
                x = info[1]
                y = info[2]
                r = tuple([(x**2 + y**2)**(1/2)])
                r2 = tuple([((x-0.054094482)**(2) + (y-0.03873037)**(2))**(1/2)])
            
                tuple1 = (z, field, theta, phi)

                data.append(tuple1 + info + r + r2 + tuple([np.nan]))
                
    return data, deleted

In [ ]:
x2, deleted2 = readradiuspkl(6, 13, 8)

In [ ]:
df2 = pd.DataFrame(x2, columns = ['z', 'field', 'theta', 'phi', 't', 'x', 'y', 'r_(0,0)', 'r_(x0,y0)', 'rreco'])

In [ ]:
df2

In [ ]:
dfnew2 = df2.dropna()
dfnew2

In [ ]:
def difarrays2(df):
    ms = []
    data = []
    missing = []
    for z in df['z'].unique():
        for field in df['field'].unique()[:11]:
            xdifs = []
            ydifs = []
            r0difs = []
            rrecodifs = []
            for theta in df['theta'].unique():
                dfnew1 = df[(df['z']==z) & np.isclose(df['theta'], theta, 1e-1) & ((df['field']=='1.00'))]
                dfnew2 = df[(df['z']==z) & np.isclose(df['theta'], theta, 1e-1) & ((df['field']==field))]                        
                if len(np.array(dfnew1['phi'])) != len(np.array(dfnew2['phi'])):
                    missing.append([z, field, theta])
                elif(np.isclose(v, 0) for v in (np.array(dfnew1['phi']) - np.array(dfnew2['phi']))):
                    xdif = np.array(dfnew2['x']) - np.array(dfnew1['x'])
                    ydif = np.array(dfnew2['y']) - np.array(dfnew1['y'])
                    r0dif = np.array(dfnew2['r_(0,0)']) - np.array(dfnew1['r_(0,0)'])
                    rrecodif = np.array(dfnew2['rreco']) - np.array(dfnew1['rreco'])
                    
                
                    for a in range(0, len(xdif), 1):
                        data.append([xdif[a], ydif[a], r0dif[a], rrecodif[a], z, field, theta])
                    for a in xdif:
                        xdifs.append(a)
                    for b in ydif:
                        ydifs.append(b)
                    for c in rrecodif:
                        rrecodifs.append(c)
                    for d in r0dif:
                        r0difs.append(d)
                        
                else:
                    print('mistake')
            ms.append([field, z, np.mean(xdifs), np.mean(ydifs), np.std(xdifs), np.std(ydifs), np.mean(rrecodifs), np.std(rrecodifs), np.mean(r0difs), np.std(r0difs)])
                
    datadf = pd.DataFrame(data, columns = ['xdif', 'ydif', 'r0dif', 'rrecodif', 'z', 'field', 'theta'])
    msdf = pd.DataFrame(ms, columns = ['field', 'z', 'xmean', 'ymean', 'xstd', 'ystd', 'rrecomean', 'rrecostd', 'r0mean', 'r0std'])
                
    return msdf, datadf, missing 

In [ ]:
msdf2, datadf2, missing2 = difarrays2(dfnew2)

In [ ]:
def makeplots2(df, z):
    fields = df['field'].unique()
    fig = plt.figure()
    ax1 = plt.subplot2grid((2,2), (0,0), rowspan=1, colspan=1)
    ax2 = plt.subplot2grid((2,2), (0,1), rowspan=1, colspan=1)
    ax3 = plt.subplot2grid((2,2), (1,0), rowspan=1, colspan=1)
    ax4 = plt.subplot2grid((2,2), (1,1), rowspan=1, colspan=1)

    df2 = df[df['z']==z]
    for field in fields:
        dfnew = df2[df2['field']==field]
        
        r0mean = list(dfnew['r0mean'])
        r0std = list(dfnew['r0std'])
        rrecomean = list(dfnew['rrecomean'])
        rrecostd = list(dfnew['rrecostd'])
        
        ax1.scatter(field, r0mean, c='r', marker='s')
        ax1.set_title(f'R0 Mean Value at Different Fields', fontsize = 20)
        ax1.set_xlabel(f'Field Map Label')
        ax1.set_ylabel(f'Mean Value (meters)')
        
        ax2.scatter(field, r0std, c='r', marker='s')
        ax2.set_title(f'R0 Std Value at Different Fields', fontsize = 20)
        ax2.set_xlabel(f'Field Map Label')
        ax2.set_ylabel(f'Std Value (meters)')
        
        ax3.scatter(field, rrecomean, c='r', marker='s')
        ax3.set_title(f'Rreco Mean Value at Different Fields', fontsize = 20)
        ax3.set_xlabel(f'Field Map Label')
        ax3.set_ylabel(f'Mean Value (meters)')
        
        ax4.scatter(field, rrecostd, c='r', marker='s')
        ax4.set_title(f'Rreco Std Value at Different Fields', fontsize = 20)
        ax4.set_xlabel(f'Field Map Label')
        ax4.set_ylabel(f'Std Value (meters)')
        
    fig.suptitle(f'Mean, Std R Displacement (Dis R - Nom R) at Z = {z}', fontsize = 30)
    fig.savefig(rad13plotdir+f'/Mean,Std_Scatter_R={z}.png')


In [ ]:
for z in zvals2:
    makeplots2(msdf2, z)

In [ ]:
for field in datadf2['field'].unique():
    fig = plt.figure()
    df1 = datadf2[(datadf2['z']==zvals2[0]) & (datadf2['field']==field)]
    df2 = datadf2[(datadf2['z']==zvals2[1]) & (datadf2['field']==field)]
    df3 = datadf2[(datadf2['z']==zvals2[2]) & (datadf2['field']==field)]
    df4 = datadf2[(datadf2['z']==zvals2[3]) & (datadf2['field']==field)]
    df5 = datadf2[(datadf2['z']==zvals2[4]) & (datadf2['field']==field)]
    df6 = datadf2[(datadf2['z']==zvals2[5]) & (datadf2['field']==field)]
    
    ax1 = plt.subplot2grid((3,2), (0,0), rowspan=1, colspan=1)
    ax1.hist(list(df1['rrecodif']), bins = 20, facecolor = 'blue')
    ax1.set_title(f'Z = {zvals2[0]}')
    ax1.set_ylabel(f'Number of Occurences')
    ax1.set_xlabel(f'Rreco Displacement (meters)')
    
    ax2 = plt.subplot2grid((3,2), (0,1), rowspan=1, colspan=1)
    ax2.hist(list(df2['rrecodif']), bins = 20, facecolor = 'blue')
    ax2.set_title(f'Z = {zvals2[1]}')
    ax2.set_ylabel(f'Number of Occurences')
    ax2.set_xlabel(f'Rreco Displacement (meters)')
    
    ax3 = plt.subplot2grid((3,2), (1,0), rowspan=1, colspan=1)
    ax3.hist(list(df3['rrecodif']), bins = 20, facecolor = 'blue')
    ax3.set_title(f'Z = {zvals2[2]}')
    ax3.set_ylabel(f'Number of Occurences')
    ax3.set_xlabel(f'Rreco Displacement (meters)')
    
    ax4 = plt.subplot2grid((3,2), (1,1), rowspan=1, colspan=1)
    ax4.hist(list(df4['rrecodif']), bins = 20, facecolor = 'blue')
    ax4.set_title(f'Z = {zvals2[3]}')
    ax4.set_ylabel(f'Number of Occurences')
    ax4.set_xlabel(f'Rreco Displacement (meters)')
    
    ax5 = plt.subplot2grid((3,2), (2,0), rowspan=1, colspan=1)
    ax5.hist(list(df5['rrecodif']), bins = 20, facecolor = 'blue')
    ax5.set_title(f'Z = {zvals2[4]}')
    ax5.set_ylabel(f'Number of Occurences')
    ax5.set_xlabel(f'Rreco Displacement (meters)')
    
    ax6 = plt.subplot2grid((3,2), (2,1), rowspan=1, colspan=1)
    ax6.hist(list(df6['rrecodif']), bins = 20, facecolor = 'blue')
    ax6.set_title(f'Z = {zvals2[5]}')
    ax6.set_ylabel(f'Number of Occurences')
    ax6.set_xlabel(f'Rreco Displacement (meters)')
    
    fig.suptitle(f'Histogram of Rreco Displacement (Dis Rreco - Nom Rreco) for Field = {field}', fontsize = 20)
    fig.tight_layout(pad = 5.0)
    fig.savefig(rad13plotdir+f'/Rreco_hist_{field}.png')

In [ ]:
for field in datadf2['field'].unique():
    fig = plt.figure()
    df1 = datadf2[(datadf2['z']==zvals2[0]) & (datadf2['field']==field)]
    df2 = datadf2[(datadf2['z']==zvals2[1]) & (datadf2['field']==field)]
    df3 = datadf2[(datadf2['z']==zvals2[2]) & (datadf2['field']==field)]
    df4 = datadf2[(datadf2['z']==zvals2[3]) & (datadf2['field']==field)]
    df5 = datadf2[(datadf2['z']==zvals2[4]) & (datadf2['field']==field)]
    df6 = datadf2[(datadf2['z']==zvals2[5]) & (datadf2['field']==field)]
    
    ax1 = plt.subplot2grid((3,2), (0,0), rowspan=1, colspan=1)
    ax1.hist(list(df1['r0dif']), bins = 20, facecolor = 'blue')
    ax1.set_title(f'Z = {zvals2[0]}')
    ax1.set_ylabel(f'Number of Occurences')
    ax1.set_xlabel(f'R0 Displacement (meters)')
    
    ax2 = plt.subplot2grid((3,2), (0,1), rowspan=1, colspan=1)
    ax2.hist(list(df2['r0dif']), bins = 20, facecolor = 'blue')
    ax2.set_title(f'Z = {zvals2[1]}')
    ax2.set_ylabel(f'Number of Occurences')
    ax2.set_xlabel(f'R0 Displacement (meters)')
    
    ax3 = plt.subplot2grid((3,2), (1,0), rowspan=1, colspan=1)
    ax3.hist(list(df3['r0dif']), bins = 20, facecolor = 'blue')
    ax3.set_title(f'Z = {zvals2[2]}')
    ax3.set_ylabel(f'Number of Occurences')
    ax3.set_xlabel(f'R0 Displacement (meters)')
    
    ax4 = plt.subplot2grid((3,2), (1,1), rowspan=1, colspan=1)
    ax4.hist(list(df4['r0dif']), bins = 20, facecolor = 'blue')
    ax4.set_title(f'Z = {zvals2[3]}')
    ax4.set_ylabel(f'Number of Occurences')
    ax4.set_xlabel(f'R0 Displacement (meters)')
    
    ax5 = plt.subplot2grid((3,2), (2,0), rowspan=1, colspan=1)
    ax5.hist(list(df5['r0dif']), bins = 20, facecolor = 'blue')
    ax5.set_title(f'Z = {zvals2[4]}')
    ax5.set_ylabel(f'Number of Occurences')
    ax5.set_xlabel(f'R0 Displacement (meters)')
    
    ax6 = plt.subplot2grid((3,2), (2,1), rowspan=1, colspan=1)
    ax6.hist(list(df6['r0dif']), bins = 20, facecolor = 'blue')
    ax6.set_title(f'Z = {zvals2[5]}')
    ax6.set_ylabel(f'Number of Occurences')
    ax6.set_xlabel(f'R0 Displacement (meters)')
    
    fig.suptitle(f'Histogram of R0 Displacement (Dis r0 - Nom r0) for Field = {field}', fontsize = 20)
    fig.tight_layout(pad = 5.0)
    fig.savefig(rad13plotdir+f'/r0_hist_{field}.png')

In [ ]:
for field in fieldvals2:
    newdf = datadf2[datadf2['field']==field]
    fig = plt.figure()
    plt.scatter(newdf['z'], newdf['r0dif'], c = 'blue')
    plt.title(f'R0 Deviations (Dis R0 - Nom R0) for Field = {field} at Different Z Values')
    plt.xlabel('Z Value (meters)')
    plt.ylabel('R0 Deviation (meters)')
    fig.savefig(rad13plotdir+f'/R0 deviations_{field}.png')

In [ ]:
for field in fieldvals2:
    newdf = datadf2[datadf2['field']==field]
    fig = plt.figure()
    plt.scatter(newdf['z'], newdf['rrecodif'], c = 'blue')
    plt.title(f'Rreco Deviations (Dis rreco - Nom rreco) for Field = {field} at Different Z Values')
    plt.xlabel('Z Value (meters)')
    plt.ylabel('Rreco Deviation (meters)')
    fig.savefig(rad13plotdir+f'/Rreco deviations_{field}.png')